In [113]:
from pygraphblas import *
from pygraphblas import lib
from pygraphblas.types import Type, binop
from pygraphblas.gviz import draw, draw_vis, draw_vector, draw_matrix
import numpy as np
import pprint
from pyvis import network as net
import networkx as nx
import scipy as sp
import copy

import this
options_set(nthreads=8)

In [114]:
class GS(Type):

    _base_name = "UDT"
    _numpy_t = None
    members = ['double w', 'uint64_t n']
    one = (float('inf'), 0)
    # equal operation is not working with "iseq" operation, D.iseq(D1)
    @binop(boolean=True)
    def EQ(z, x, y):
        if x.w == y.w and x.n == y.n:
            z = True
        else:
            z = False

    @binop()
    def PLUS(z, x, y):
        if x.w < y.w:
            z.w = x.w
            z.n = x.n
        elif x.w == y.w:
            z.w = x.w
            z.n = x.n + y.n
        else:
            z.w = y.w
            z.n = y.n

    @binop()
    def TIMES(z, x, y):
        z.w = x.w + y.w
        z.n = x.n * y.n

GS_monoid = GS.new_monoid(GS.PLUS, GS.one)
GS_semiring = GS.new_semiring(GS_monoid, GS.TIMES)

def shortest_path(matrix, start):
    n = matrix.nrows
    v = Vector.sparse(matrix.type, n)
    for i, j, k in matrix:
        if i == j:
            matrix[i,j] = (float('inf'), 0)
        else:
            matrix[i,j] = (k[0], 1)        
    v = matrix.extract_row(start)
    v_k = matrix.extract_row(start)
    with GS_semiring:
        for _ in range(matrix.nrows + 1):
            v_k = v_k @ matrix
            v = v + v_k
    return v

A = Matrix.sparse(GS, 6, 6)
A[0,1] = (9.0, 1)
A[0,3] = (3.0, 1)
A[0,5] = (4.0, 1)
A[1,2] = (8.0, 1)
A[3,4] = (6.0, 1)
A[3,5] = (1.0, 1)
A[4,2] = (4.0, 1)
A[1,5] = (7.0, 1)
A[5,4] = (2.0, 1)

N = net.Network(notebook=True, directed=True)
for i, j, v in A:
    N.add_node(i, label=str(i), shape='circle')
    N.add_node(j, label=str(j), shape='circle')
    N.add_edge(i, j, label=str(v[0]))
N.show('graph.html')

graph.html


In [115]:
g = draw(A, label_vector=shortest_path(A, 0))

In [116]:
def shortest_path_FW(matrix):
    n = matrix.nrows
    v = Vector.sparse(matrix.type, n)
    for i, j, k in matrix:
        if i == j:
            matrix[i,j] = (float('inf'), 0)
        else:
            matrix[i,j] = (k[0], 1)        
    D_k = matrix.dup()
    D = matrix.dup()
    with GS_semiring:
        for _ in range(matrix.nrows):
            D_k @= matrix
            D += D_k
    return D

In [117]:
D = shortest_path_FW(A)

In [118]:
for i, j, k in D:
    print(i, j, D[i, j])

0 1 (9.0, 1)
0 2 (10.0, 2)
0 3 (3.0, 1)
0 4 (6.0, 2)
0 5 (4.0, 2)
1 2 (8.0, 1)
1 4 (9.0, 1)
1 5 (7.0, 1)
3 2 (7.0, 1)
3 4 (3.0, 1)
3 5 (1.0, 1)
4 2 (4.0, 1)
5 2 (6.0, 1)
5 4 (2.0, 1)


In [119]:
N = net.Network(notebook=True, directed=True, height=300)
labels = {}
for i, j, v in D:
    if j not in labels:
        labels[j] = [f"({i},{j},{v[0]},{v[1]})"]
    else:
        labels[j].append(f"({i},{j},{v[0]},{v[1]})")
for i, j, v in A:
    if i in labels:
        N.add_node(i, label='\n'.join(labels[i]), shape='circle')
    else:
        N.add_node(i, label=str(i), shape='circle')
    if j in labels:
        N.add_node(j, label='\n'.join(labels[j]), shape='circle')
    else:
        N.add_node(j, label=str(j), shape='circle')
    N.add_edge(i, j, label=str(v))     
N.show('graph.html')

graph.html


In [120]:
def algebraic_IPC_with_paths(Adj, x):
    n = Adj.shape[0]
    A = Matrix.sparse(GS, n, n)
    A1 = Matrix.sparse(FP64, n, n)
    w_s_r = 0
    w_v = np.zeros(n)
    # compute relative contribution coefficients (weights)
    for s in range(n):
        for r in range(n):
            if x[r] - x[s] > 0:
                w_s_r += x[r] - x[s]
                w_v[s] += x[r] - x[s]
                w_v[r] += x[r] - x[s]
    for k, v in Adj.items():
        i = k[0]
        j = k[1]
        if i == j:
            A[i, j] = (float('inf'), 0)
        else:
            A[i, j] = (v, 1)
            A1[i, j] = v
    D = shortest_path_FW(A)
    D1 = Matrix.sparse(FP64, n, n)
    for i, j, v in D:
        D1[i, j] = v[0]
    pred = dict()
    with FP64.plus:
        for i in range(n):
            pred[i] = dict()
            d = D1.extract_row(i)
            d[i] = 0
            for j in d.indices:
                col_j = A1.extract_col(j)
                col_j.emult(d, mult_op=FP64.plus, out=col_j)
                #pred_j = list((col_j == d[j]).indices)
                #if pred_j:
                pred[i][j] = list((col_j == d[j]).indices)
    #IPC = Vector.sparse(FP64, n)
    IPC = np.zeros(n)
    paths = dict()
    v_paths = dict()
    for v in range(n):
        v_paths[v] = []
        for s in D.extract_col(v).indices:
            for r in D.extract_row(v).indices:
                if x[r] - x[s] <= 0 or D[s, r][1] == 0:
                    continue
                if s != v and r != v and s != r and D[s, r][0] == D[s, v][0] + D[v, r][0]:
                    w_v_sr = (x[r] - x[s]) / (w_s_r - w_v[v]) 
                    '''
                    if (s, t) not in paths:
                        paths[(s,t)] = st_paths(pred[s], s, t)
                    for p in paths[(s, t)]:
                        if v in p:
                            v_paths[v].append(p)
                            V.update(p)
                    '''
                    if (s, r) not in paths:
                        paths[(s, r)] = st_paths(pred[s], s, r)
                    for p in paths[(s, r)]:
                        if v in p:
                            v_paths[v].append(p)
                    IPC[v] += D[s, v][1] * D[v, r][1] / D[s, r][1] * w_v_sr
    return IPC, v_paths, paths, pred


def st_paths(pred, s, t):
    stack = [[t, 0]]
    top = 0
    paths = []
    while top >= 0:
        node, i = stack[top]
        if node == s:
            paths.append([p for p, n in reversed(stack[:top+1])])
        if node not in pred:
            continue
        if len(pred[node]) > i:
            top += 1
            if top == len(stack):
                stack.append([pred[node][i],0])
            else:
                stack[top] = [pred[node][i],0]
        else:
            stack[top-1][1] += 1
            top -= 1
    return paths

In [121]:
def IPC_with_paths_nx(G, x):
    '''
    Inverse percolation centrality algorithm using Networkx
    '''
    #IPC = dict.fromkeys(G, 0.0)
    n = G.number_of_nodes()
    print('NetworkX graph weights')
    for u, v, weight in G.edges(data="weight"):
        print(u, v, weight)
    IPC = np.zeros(n)
    w_s_r = 0
    w_v = np.zeros(n)
    # compute relative contribution coefficients (weights)
    for s in range(n):
        for r in range(n):
            if x[r] - x[s] > 0:
                w_s_r += x[r] - x[s]
                w_v[s] += x[r] - x[s]
                w_v[r] += x[r] - x[s]
    D = dict(nx.all_pairs_dijkstra(G, weight='weight'))
    v_paths = dict()
    paths = dict()
    for v in range(n):
        if v not in D:
            continue
        for s in D:
            if v not in D[s][0]:
                continue
            for r in D[s][0]:
                if r not in D[v][0]:
                    continue
                if x[r] - x[s] <= 0:
                    continue
                if s != v and r != v and s != r and D[s][0][r] == D[s][0][v] + D[v][0][r]:
                    if (s, r) not in paths:
                        paths[(s, r)] = list(nx.all_shortest_paths(G, source=s, target=r, weight='weight'))
                    sigma_sr = len(paths[(s, r)])
                    if sigma_sr == 0:
                        continue
                    sv_paths = list(nx.all_shortest_paths(G, source=s, target=v, weight='weight'))
                    vr_paths = list(nx.all_shortest_paths(G, source=v, target=r, weight='weight'))
                    sigma_v_sr = float(len(sv_paths) * len(vr_paths))
                    w_v_sr = (x[r] - x[s]) / (w_s_r - w_v[v]) 
                    if v not in v_paths:
                        v_paths[v] = []
                    IPC[v] += sigma_v_sr / sigma_sr * w_v_sr
                    for p in paths[(s, r)]:
                        if v in p:
                            v_paths[v].append(p)
    return IPC, v_paths, paths

In [122]:
def IPC_dense(G, x):
    '''
    Inverse centrality algorithm
    '''
    # adjacency matrix
    A = nx.adjacency_matrix(G)
    A = A.todok(copy=True)
    n = A.shape[0]
    # centrality
    C = np.zeros(n)
    # shortest path distance matrix
    D_d = np.zeros((n, n), dtype=float)
    D_d[:, :] = float('inf')
    # shortest path count matrix
    D_s = np.zeros((n, n), dtype=float)
    # initiate D_d and D_s
    for i in range(n):
        D_d[i, i] = 0
        D_s[i, i] = 1
    for k, v in A.items():
        i = k[0]
        j = k[1]
        if i != j and v != 0: 
            D_d[i, j] = v
            D_s[i, j] = 1
    w_s_r = 0
    w_v = np.zeros(n)
    # compute relative contribution coefficients (weights)
    for s in range(n):
        for r in range(n):
            if x[r] - x[s] > 0:
                w_s_r += x[r] - x[s]
                w_v[s] += x[r] - x[s]
                w_v[r] += x[r] - x[s]
    # iterate for each vertex
    for k in range(n):
        # iterate for each vertex
        for i in range(n):
            # iterate for each vertex
            for j in range(n):
                # shortest paths through k exists
                if D_d[i, k] + D_d[k, j] < D_d[i, j]:
                    D_d[i, j] = D_d[i, k] + D_d[k, j]
                    D_s[i, j] = D_s[i, k] * D_s[k, j]
                # other shortest paths through k exist
                elif D_d[i, k] + D_d[k, j] == D_d[i, j] and i != j:
                    D_s[i, j] += D_s[i, k] * D_s[k, j]
    # iterate for each vertex
    for v in range(n):
        # iterate for each vertex
        for s in range(n):
            # iterate for each vertex
            for r in range(n):
                if s != v and r != v and D_d[s, r] == D_d[s, v] + D_d[v, r] and x[r] - x[s] > 0:
                    # the weight
                    w_v_sr = (x[r] - x[s]) / (w_s_r - w_v[v])
                    # accummulate centrality value
                    C[v] += (D_s[s, v] * D_s[v, r]) / D_s[s, r] * w_v_sr
    return C


def IPC_sparse(G, x):
    '''
    Inverse centrality algorithm
    '''
    # adjacency matrix
    A = nx.adjacency_matrix(G)
    A = A.todok(copy=True)
    n = A.shape[0]
    # centrality
    C = np.zeros(n)
    # shortest path distance matrix
    D_d = copy.deepcopy(A)
    # shortest path count matrix
    D_s = sp.sparse.dok_matrix((n, n), dtype=float)
    for i in range(n):
        D_s[i, i] = 1
        D_d[i, i] = 0
    for k, v in A.items():
        i = k[0]
        j = k[1]
        if i != j and v != 0: 
            D_s[i, j] = 1
    w_s_r = 0
    w_v = np.zeros(n)
    # compute relative contribution coefficients (weights)
    for s in range(n):
        for r in range(n):
            if x[r] - x[s] > 0:
                w_s_r += x[r] - x[s]
                w_v[s] += x[r] - x[s]
                w_v[r] += x[r] - x[s]
    # iterate for each vertex
    for k in range(n):
        # if k-th col is empty, continue
        if D_d.getcol(k).count_nonzero() == 0:
            continue
        # iterate over nonzero column indices
        for i in D_d.getcol(k).nonzero()[0]:
            if D_d.getrow(k).count_nonzero() == 0:
                continue
            # iterate over nonzero row indices
            for j in D_d.getrow(k).nonzero()[1]:
                # check the entry for i, j
                if not D_d.get((i, j), False):
                    d_ij = float('inf')
                else:
                    d_ij = D_d[i, j]
                # shortest paths through k exists
                if D_d[i, k] + D_d[k, j] < d_ij:
                    D_d[i, j] = D_d[i, k] + D_d[k, j]
                    D_s[i, j] = D_s[i, k] * D_s[k, j]
                # other shortest paths through k exist
                elif D_d[i, k] + D_d[k, j] == d_ij and i != j:
                    D_s[i, j] += D_s[i, k] * D_s[k, j]
    for v in range(n):
        # if v-th col is empty, continue
        if D_d.getcol(v).count_nonzero() == 0:
            continue
        # iterate over nonzero column indices
        for s in D_d.getcol(v).nonzero()[0]:
            # if v-th row is empty, continue
            if D_d.getrow(v).count_nonzero() == 0:
                continue
            # iterate over nonzero row indices
            for r in D_d.getrow(v).nonzero()[1]:
                if s == r or s == v or r == v or x[r] - x[s] <= 0:
                    continue
                if not D_d.get((s, v), False) or not D_d.get((v, r), False):
                    continue
                if D_d[s, r] == D_d[s, v] + D_d[v, r]:
                    # the weight
                    w_v_sr = (x[r] - x[s]) / (w_s_r - w_v[v])
                    # accummulate centrality value
                    C[v] += (D_s[s, v] * D_s[v, r]) / D_s[s, r] * w_v_sr
    return C

In [112]:
from scipy.sparse import dok_matrix

Adj = dok_matrix((6, 6), dtype=float) 
Adj[0, 1] = 9.0
Adj[0, 3] = 3.0
Adj[0, 5] = 4.0
Adj[1, 2] = 8.0
Adj[3, 4] = 6.0
Adj[3, 5] = 1.0
Adj[4, 2] = 4.0
Adj[1, 5] = 7.0
Adj[5, 4] = 2.0

states = np.array([0, 0, 0.4, 0, 0, 0])

IPC, v_paths, paths, pred = algebraic_IPC_with_paths(Adj, states)
print('Algebraic IPC', IPC)
#print(v_paths)
G = nx.from_numpy_array(Adj, create_using=nx.DiGraph)
C_nx, v_paths_nx, paths_nx = IPC_with_paths_nx(G, x=states)
print('C_nx', C_nx)
#print(v_paths_nx)
C_dense = IPC_dense(G, states)
C_sparse = IPC_sparse(G, states)
#print(C_dense)
print('C_sparse', C_sparse)

Algebraic IPC [0.    0.    0.    0.125 0.75  0.5  ]
NetworkX graph weights
0 1 9.0
0 3 3.0
0 5 4.0
1 2 8.0
1 5 7.0
3 4 6.0
3 5 1.0
4 2 4.0
5 4 2.0
C_nx [0.    0.    0.    0.125 0.75  0.5  ]
C_sparse [0.    0.    0.    0.125 0.75  0.5  ]


/var/folders/vz/4_nwbcw97xjdd1y7_nfhyb_80000gp/T/ipykernel_56878/1561029555.py:6: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G)
/var/folders/vz/4_nwbcw97xjdd1y7_nfhyb_80000gp/T/ipykernel_56878/1561029555.py:67: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G)


In [110]:
print(pred)

{0: {0: [], 1: [0], 2: [4], 3: [0], 4: [5], 5: [0, 3]}, 1: {1: [], 2: [1], 4: [5], 5: [1]}, 2: {2: []}, 3: {2: [4], 3: [], 4: [5], 5: [3]}, 4: {2: [4], 4: []}, 5: {2: [4], 4: [5], 5: []}}


In [123]:
v = 0
if v in pred:
    N = net.Network(notebook=True, directed=True, height=300)
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[v]:
            continue
        if i in pred[v][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp.html')


sssp.html


In [124]:
v = 1
if v in pred:
    N = net.Network(notebook=True, directed=True, height=300)
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[v]:
            continue
        if i in pred[v][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp.html')

sssp.html


In [125]:
v = 2
if v in pred:
    N = net.Network(notebook=True, directed=True, height=300)
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[v]:
            continue
        if i in pred[v][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp.html')

sssp.html


In [126]:
v = 3
if v in pred:
    N = net.Network(notebook=True, directed=True, height=300)
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[v]:
            continue
        if i in pred[v][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp.html')

sssp.html


In [54]:
v = 4
if v in pred:
    N = net.Network(notebook=True, directed=True, height=300)
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[v]:
            continue
        if i in pred[v][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp.html')

sssp.html


In [55]:
v = 5
if v in pred:
    N = net.Network(notebook=True, directed=True, height=300)
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[v]:
            continue
        if i in pred[v][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp.html')

sssp.html
